In [1]:
import os
import sys

MODULE_PATH = '/content/drive/MyDrive/GitHub/DL_Study/Base'

sys.path.insert(0, MODULE_PATH)
sys.path

['/content/drive/MyDrive/GitHub/DL_Study/Base',
 '',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython']

In [2]:
# import
import numpy
from config import *
from optim import Adam
from models import CnnModelReg

# for time series split
!pip install scikit-learn==0.24.2

     |████████████████████████████████| 22.3MB 1.4MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [3]:
# configuration setting
def model_config():
    # parameter for CNN Model
    filter_num = [30]
    filter_size = [3]
    epochs = [30]
    batch_size = [64]
    learning_rate = [0.01, 0.001]
    
    # create config data
    configs = []
    for i in filter_num:
        for j in filter_size:
            for k in epochs:
                for l in batch_size:
                    for m in learning_rate:
                        config = [i, j, k, l, m]
                        configs.append(config)
    return configs

# fucntion for fit cnn model using configs
def model_fit(train_X, train_y, config):
    # unpack config
    n_filter, n_fsize, n_epochs, n_batch, learning_rate = config
    cnn_params = {'filter_num':n_filter, 'filter_size':n_fsize, 
                  'stride':1, 'padding':0}
    model = CnnModelReg(input_dim=(1, 24, 20),params=cnn_params)
    # fit model and return
    model.fit(train_X=train_X, train_y=train_y, epochs=n_epochs, 
              batch_size=n_batch, learning_rate=learning_rate)
    return model

def MAE_metric(x, t):
    t = np.array(t)
    return np.mean(numpy.abs(x-t))

def MSE_metric(x, t):
    t = np.array(t)
    return np.mean((x-t)**2)

In [4]:
import zipfile, requests, io
import pandas as pd
import numpy
import time
from datetime import datetime

data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00580/hungary_chickenpox.zip'
r = requests.get(data_url)
files = zipfile.ZipFile(io.BytesIO(r.content))
df = pd.read_csv(files.open('hungary_chickenpox.csv'), sep=',')
df.drop('Date', axis=1, inplace=True)
df.head()

,BUDAPEST,BARANYA,BACS,BEKES,BORSOD,CSONGRAD,FEJER,GYOR,HAJDU,HEVES,JASZ,KOMAROM,NOGRAD,PEST,SOMOGY,SZABOLCS,TOLNA,VAS,VESZPREM,ZALA
0,168,79,30,173,169,42,136,120,162,36,130,57,2,178,66,64,11,29,87,68
1,157,60,30,92,200,53,51,70,84,28,80,50,29,141,48,29,58,53,68,26
2,96,44,31,86,93,30,93,84,191,51,64,46,4,157,33,33,24,18,62,44
3,163,49,43,126,46,39,52,114,107,42,63,54,14,107,66,50,25,21,43,31
4,122,78,53,87,103,34,95,131,172,40,61,49,11,124,63,56,7,47,85,60


In [5]:
from scipy.stats import skew, kurtosis
from statsmodels.tsa.stattools import adfuller

# jb = (n/6)*(skewness**2 + (kurtosis**2/4))

def data_statistics(df):
    df = df.dropna()
    data = df.values
    num = len(df)
    skewness_ = skew(data)
    kurtosis_ = kurtosis(data)
    jarque_bera_ = (num/6)*(skewness_**2 + (kurtosis_**2/4))
    result = adfuller(data)
    adf_ = result[0]
    print(f'skewness : {skewness_}')
    print(f'kurtosis : {kurtosis_}')
    print(f'jarque bera : {jarque_bera_}')
    print(f'ADF : {adf_}')

data_statistics(df['BUDAPEST'])

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


skewness : 0.9489236649041097
kurtosis : 1.4006683128498736
jarque bera : 121.01039256493132
ADF : -6.933218014746711


In [6]:
df.describe()

,BUDAPEST,BARANYA,BACS,BEKES,BORSOD,CSONGRAD,FEJER,GYOR,HAJDU,HEVES,JASZ,KOMAROM,NOGRAD,PEST,SOMOGY,SZABOLCS,TOLNA,VAS,VESZPREM,ZALA
count,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000
mean,101.245211,34.204981,37.166667,28.911877,57.082375,31.488506,33.272031,41.436782,47.097701,29.691571,40.869732,25.643678,21.850575,86.101533,27.609195,29.854406,20.352490,22.467433,40.636015,19.873563
std,76.354872,32.567222,36.843095,37.618092,50.725437,33.790208,31.397989,36.014297,44.610836,31.857750,37.283299,24.467995,22.025999,66.773741,26.724236,31.814630,23.273025,25.006638,40.699471,21.999636
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,34.250000,8.000000,8.000000,4.000000,14.250000,6.000000,7.000000,9.000000,11.000000,6.250000,10.000000,6.000000,4.000000,28.250000,6.000000,6.000000,4.000000,3.000000,7.250000,4.000000
50%,93.000000,25.000000,29.500000,14.000000,46.500000,20.500000,24.000000,35.000000,37.000000,21.000000,31.000000,19.000000,15.000000,81.000000,20.500000,18.500000,12.000000,13.000000,32.000000,13.000000
75%,149.000000,51.000000,53.000000,38.750000,83.750000,47.000000,51.750000,63.000000,68.000000,41.000000,61.750000,39.000000,32.750000,129.750000,41.000000,45.000000,29.000000,34.000000,59.000000,31.000000
max,479.000000,194.000000,274.000000,271.000000,355.000000,199.000000,164.000000,181.000000,262.000000,210.000000,224.000000,160.000000,112.000000,431.000000,155.000000,203.000000,131.000000,141.000000,230.000000,216.000000


In [7]:
df.isnull().sum()

BUDAPEST    0
BARANYA     0
BACS        0
BEKES       0
BORSOD      0
CSONGRAD    0
FEJER       0
GYOR        0
HAJDU       0
HEVES       0
JASZ        0
KOMAROM     0
NOGRAD      0
PEST        0
SOMOGY      0
SZABOLCS    0
TOLNA       0
VAS         0
VESZPREM    0
ZALA        0
dtype: int64

In [8]:
# series data to img function
def series_to_img(dataset, time_step=1):
    num = dataset.shape[1]      # features num
    df = pd.DataFrame(dataset)
    cols, names = list(), list()
    # sequence t-n to t-1
    for i in range(time_step, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(num)]

    for i in range(0, 1):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(num)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(num)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler

dataset = df.values
dataset = dataset.astype('float')

n_inputs = 24
n_features = 20
del_idx = n_inputs * n_features + 1
del_cols = [i for i in range(del_idx, del_idx+n_features-1)]
new_df = series_to_img(dataset, n_inputs)
new_df.drop(new_df.columns[del_cols], axis=1, inplace=True)
new_df.head()

,var1(t-24),var2(t-24),var3(t-24),var4(t-24),var5(t-24),var6(t-24),var7(t-24),var8(t-24),var9(t-24),var10(t-24),var11(t-24),var12(t-24),var13(t-24),var14(t-24),var15(t-24),var16(t-24),var17(t-24),var18(t-24),var19(t-24),var20(t-24),var1(t-23),var2(t-23),var3(t-23),var4(t-23),var5(t-23),var6(t-23),var7(t-23),var8(t-23),var9(t-23),var10(t-23),var11(t-23),var12(t-23),var13(t-23),var14(t-23),var15(t-23),var16(t-23),var17(t-23),var18(t-23),var19(t-23),var20(t-23),...,var2(t-2),var3(t-2),var4(t-2),var5(t-2),var6(t-2),var7(t-2),var8(t-2),var9(t-2),var10(t-2),var11(t-2),var12(t-2),var13(t-2),var14(t-2),var15(t-2),var16(t-2),var17(t-2),var18(t-2),var19(t-2),var20(t-2),var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var10(t-1),var11(t-1),var12(t-1),var13(t-1),var14(t-1),var15(t-1),var16(t-1),var17(t-1),var18(t-1),var19(t-1),var20(t-1),var1(t)
24,168.0,79.0,30.0,173.0,169.0,42.0,136.0,120.0,162.0,36.0,130.0,57.0,2.0,178.0,66.0,64.0,11.0,29.0,87.0,68.0,157.0,60.0,30.0,92.0,200.0,53.0,51.0,70.0,84.0,28.0,80.0,50.0,29.0,141.0,48.0,29.0,58.0,53.0,68.0,26.0,...,98.0,129.0,100.0,104.0,48.0,73.0,128.0,95.0,62.0,163.0,146.0,39.0,145.0,23.0,84.0,46.0,39.0,87.0,37.0,139.0,30.0,94.0,54.0,61.0,30.0,56.0,79.0,161.0,83.0,99.0,82.0,29.0,66.0,13.0,14.0,23.0,29.0,69.0,47.0,155.0
25,157.0,60.0,30.0,92.0,200.0,53.0,51.0,70.0,84.0,28.0,80.0,50.0,29.0,141.0,48.0,29.0,58.0,53.0,68.0,26.0,96.0,44.0,31.0,86.0,93.0,30.0,93.0,84.0,191.0,51.0,64.0,46.0,4.0,157.0,33.0,33.0,24.0,18.0,62.0,44.0,...,30.0,94.0,54.0,61.0,30.0,56.0,79.0,161.0,83.0,99.0,82.0,29.0,66.0,13.0,14.0,23.0,29.0,69.0,47.0,155.0,56.0,94.0,56.0,147.0,24.0,77.0,93.0,150.0,47.0,123.0,83.0,30.0,117.0,36.0,73.0,21.0,53.0,66.0,70.0,98.0
26,96.0,44.0,31.0,86.0,93.0,30.0,93.0,84.0,191.0,51.0,64.0,46.0,4.0,157.0,33.0,33.0,24.0,18.0,62.0,44.0,163.0,49.0,43.0,126.0,46.0,39.0,52.0,114.0,107.0,42.0,63.0,54.0,14.0,107.0,66.0,50.0,25.0,21.0,43.0,31.0,...,56.0,94.0,56.0,147.0,24.0,77.0,93.0,150.0,47.0,123.0,83.0,30.0,117.0,36.0,73.0,21.0,53.0,66.0,70.0,98.0,40.0,153.0,32.0,85.0,13.0,31.0,62.0,40.0,42.0,82.0,81.0,10.0,89.0,12.0,60.0,22.0,25.0,49.0,18.0,87.0
27,163.0,49.0,43.0,126.0,46.0,39.0,52.0,114.0,107.0,42.0,63.0,54.0,14.0,107.0,66.0,50.0,25.0,21.0,43.0,31.0,122.0,78.0,53.0,87.0,103.0,34.0,95.0,131.0,172.0,40.0,61.0,49.0,11.0,124.0,63.0,56.0,7.0,47.0,85.0,60.0,...,40.0,153.0,32.0,85.0,13.0,31.0,62.0,40.0,42.0,82.0,81.0,10.0,89.0,12.0,60.0,22.0,25.0,49.0,18.0,87.0,18.0,38.0,26.0,69.0,32.0,33.0,66.0,106.0,40.0,84.0,72.0,10.0,91.0,11.0,86.0,12.0,54.0,51.0,34.0,110.0
28,122.0,78.0,53.0,87.0,103.0,34.0,95.0,131.0,172.0,40.0,61.0,49.0,11.0,124.0,63.0,56.0,7.0,47.0,85.0,60.0,174.0,76.0,77.0,152.0,189.0,26.0,74.0,181.0,157.0,44.0,95.0,97.0,26.0,146.0,59.0,54.0,27.0,54.0,48.0,60.0,...,18.0,38.0,26.0,69.0,32.0,33.0,66.0,106.0,40.0,84.0,72.0,10.0,91.0,11.0,86.0,12.0,54.0,51.0,34.0,110.0,15.0,67.0,13.0,36.0,20.0,24.0,41.0,49.0,18.0,58.0,43.0,12.0,31.0,11.0,35.0,8.0,18.0,39.0,14.0,70.0


In [9]:
n_splits = 3
test_size = (int)(len(new_df)*0.2)
train_test_split = TimeSeriesSplit(n_splits=n_splits+1, gap=n_inputs, test_size=test_size).split(new_df)
next(train_test_split)

configs = model_config()
history = []

best_rmse, best_mse, best_mae = [], [], []

i = 1

print('config : filter_num, filter_size, epochs, batch_size, learning_rate')

# nested cross validation for time series model
for train_cv_indices, test_cv_indices in train_test_split:
    print(f'fold : {i}/{n_splits}')
    i+=1

    # split x, y data
    train_cv_X, train_cv_y = new_df.iloc[train_cv_indices, :-1].values, new_df.iloc[train_cv_indices,-1].values
    test_cv_X, test_cv_y = new_df.iloc[test_cv_indices, :-1].values, new_df.iloc[test_cv_indices, -1].values

    # length for validation set
    test_length = len(test_cv_X)

    # scaling data
    scaler_x = MinMaxScaler()
    train_cv_X = scaler_x.fit_transform(train_cv_X)
    test_cv_X = scaler_x.transform(test_cv_X)

    train_X, val_X = train_cv_X[:-test_length, :], train_cv_X[-test_length:, :]
    train_y, val_y = train_cv_y[:-test_length], train_cv_y[-test_length:]

    # reshape
    # inner loop
    train_X = train_X.reshape(-1, 1, n_inputs, n_features)
    val_X = val_X.reshape(-1, 1, n_inputs, n_features)
    train_y = train_y.reshape(-1, 1)
    val_y = val_y.reshape(-1, 1)

    # outer loop
    train_cv_X = train_cv_X.reshape(-1, 1, n_inputs, n_features)
    test_cv_X = test_cv_X.reshape(-1, 1, n_inputs, n_features)
    train_cv_y = train_cv_y.reshape(-1, 1)
    test_cv_y = test_cv_y.reshape(-1, 1)

    # model fit, inner
    errors = []
    for idx, cfg in enumerate(configs):
        print(f' == train {cfg} model == ', end=' ')
        model = model_fit(train_X, train_y, cfg)
        predicted = model.predict(val_X)
        error = np.sqrt(MSE_metric(predicted, val_y))   # rmse
        print(f'error(rmse):{error.item():.2f}')
        if errors:
            if error < min(errors):
                param = idx
        else:
            param = idx
        errors.append(error)

    history.append(errors)

    # outer
    selected_model = model_fit(train_cv_X,train_cv_y, configs[param])
    predicted = selected_model.predict(test_cv_X)
    rmse = np.sqrt(MSE_metric(predicted, test_cv_y))
    mse = MSE_metric(predicted, test_cv_y)
    mae = MAE_metric(predicted, test_cv_y)
    best_rmse.append(rmse)
    best_mse.append(mse)
    best_mae.append(mae)

    # model eval
    print(f'train-size:{train_X.shape[0]}, val-size:{val_X.shape[0]}, test-size:{test_cv_X.shape[0]}')
    print(f'best_model => error(rmse) : {rmse.item():.2f}, param:{configs[param]}')
    print()

config : filter_num, filter_size, epochs, batch_size, learning_rate
fold : 1/3
 == train [30, 3, 30, 64, 0.01] model ==  error(rmse):63.06
 == train [30, 3, 30, 64, 0.001] model ==  error(rmse):87.83
train-size:78, val-size:99, test-size:99
best_model => error(rmse) : 44.75, param:[30, 3, 30, 64, 0.01]

fold : 2/3
 == train [30, 3, 30, 64, 0.01] model ==  error(rmse):41.32
 == train [30, 3, 30, 64, 0.001] model ==  error(rmse):57.70
train-size:177, val-size:99, test-size:99
best_model => error(rmse) : 121.80, param:[30, 3, 30, 64, 0.01]

fold : 3/3
 == train [30, 3, 30, 64, 0.01] model ==  error(rmse):51.18
 == train [30, 3, 30, 64, 0.001] model ==  error(rmse):58.20
train-size:276, val-size:99, test-size:99
best_model => error(rmse) : 50.27, param:[30, 3, 30, 64, 0.01]



In [10]:
predicted = selected_model.predict(test_cv_X)

def model_evaluation(mse, rmse, mae):
    mse = np.array(mse)
    rmse = np.array(rmse)
    mae = np.array(mae)
    print(f'MSE: mean={np.mean(mse)}, std={np.std(mse)}')
    print(f'RMSE: mean={np.mean(rmse)}, std={np.std(rmse)}')
    print(f'MAE: mean={np.mean(mae)}, std={np.std(mae)}')

model_evaluation(best_mse, best_rmse, best_mae)

MSE: mean=6454.813563092437, std=5929.4246346492
RMSE: mean=72.27268806845522, std=35.09233709019971
MAE: mean=55.613905248652664, std=30.43803167687203
